In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

In [2]:
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep = ';')
y = df.pop('quality')

In [3]:
for i in df.columns:
    df[i] = df[i].fillna(np.mean(df[i]))
train, test, y_train, y_test = train_test_split(df, y, test_size = 0.2)

In [4]:
lr = LogisticRegression()
lr.fit(train, y_train)
y_pred = lr.predict(test)
print('Accuracy score baseline:', accuracy_score(y_test, y_pred))

Accuracy score baseline: 0.514285714286


In [5]:
def fit_predict(train, test, y_train, y_test, scaler, max_depth, 
                criterion = 'entropy', max_features = 1, min_samples_split = 4):
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)        
    dt = DecisionTreeClassifier(criterion = criterion, max_depth=max_depth, 
                                random_state=42, max_features=max_features,
                               min_samples_split=min_samples_split)
    dt.fit(train_scaled, y_train)
    y_pred = dt.predict(test_scaled)
    print(accuracy_score(y_test, y_pred))

### Max depth tuning

In [6]:
for i in range(1, 20):
    print('Accuracy score using max_depth =', i, end = ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), i)

Accuracy score using max_depth = 1: 0.440816326531
Accuracy score using max_depth = 2: 0.440816326531
Accuracy score using max_depth = 3: 0.45306122449
Accuracy score using max_depth = 4: 0.460204081633
Accuracy score using max_depth = 5: 0.486734693878
Accuracy score using max_depth = 6: 0.460204081633
Accuracy score using max_depth = 7: 0.497959183673
Accuracy score using max_depth = 8: 0.50306122449
Accuracy score using max_depth = 9: 0.518367346939
Accuracy score using max_depth = 10: 0.49693877551
Accuracy score using max_depth = 11: 0.514285714286
Accuracy score using max_depth = 12: 0.492857142857
Accuracy score using max_depth = 13: 0.575510204082
Accuracy score using max_depth = 14: 0.571428571429
Accuracy score using max_depth = 15: 0.533673469388
Accuracy score using max_depth = 16: 0.548979591837
Accuracy score using max_depth = 17: 0.548979591837
Accuracy score using max_depth = 18: 0.577551020408
Accuracy score using max_depth = 19: 0.561224489796


### Max features tuning

In [7]:
for i in np.arange(0.1, 1.0, 0.1):
    print('Accuracy score using max features =', i, end = ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), max_depth = 18, max_features=i)

Accuracy score using max features = 0.1: 0.577551020408
Accuracy score using max features = 0.2: 0.608163265306
Accuracy score using max features = 0.3: 0.608163265306
Accuracy score using max features = 0.4: 0.595918367347
Accuracy score using max features = 0.5: 0.60612244898
Accuracy score using max features = 0.6: 0.580612244898
Accuracy score using max features = 0.7: 0.60306122449
Accuracy score using max features = 0.8: 0.572448979592
Accuracy score using max features = 0.9: 0.576530612245


### Min samples split tuning

In [8]:
for i in range(2, 10):
    print('Accuracy score using min samples split =', i, end = ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 18, max_features=0.3, min_samples_split=i)

Accuracy score using min samples split = 2: 0.613265306122
Accuracy score using min samples split = 3: 0.592857142857
Accuracy score using min samples split = 4: 0.608163265306
Accuracy score using min samples split = 5: 0.561224489796
Accuracy score using min samples split = 6: 0.580612244898
Accuracy score using min samples split = 7: 0.563265306122
Accuracy score using min samples split = 8: 0.595918367347
Accuracy score using min samples split = 9: 0.55612244898


### Criterion tuning

In [9]:
for i in ['gini', 'entropy']:
    print('Accuracy score using criterion =', i, end = ': ')
    fit_predict(train, test, y_train, y_test, StandardScaler(), 18, 
                max_features=0.3, min_samples_split=2, criterion = 'entropy')

Accuracy score using criterion = gini: 0.613265306122
Accuracy score using criterion = entropy: 0.613265306122


In [10]:
def create_poly(train,test,degree):
    poly = PolynomialFeatures(degree=degree)
    train_poly = poly.fit_transform(train)
    test_poly = poly.fit_transform(test)
    return train_poly,test_poly

In [11]:
for degree in [1,2,3,4]:
    train_poly, test_poly = create_poly(train, test, degree)
    print('Polynomial degree',degree)
    fit_predict(train_poly, test_poly, y_train, y_test, StandardScaler(), 18, 
                max_features=0.3, min_samples_split=2, criterion = 'entropy')
    print(10*'-')
    
train_poly, test_poly = create_poly(train, test, 2) 

Polynomial degree 1
0.604081632653
----------
Polynomial degree 2
0.625510204082
----------
Polynomial degree 3
0.625510204082
----------
Polynomial degree 4
0.609183673469
----------


In [12]:
def feat_eng(df):
    df['eng1'] = df['fixed acidity'] * df['pH']
    df['eng2'] = df['total sulfur dioxide'] / df['free sulfur dioxide']
    df['eng3'] = df['sulphates'] / df['chlorides']
    df['eng4'] = df['chlorides'] / df['sulphates']
    return df

train = feat_eng(train)
test = feat_eng(test)

print('Additional feature engineering:')

fit_predict(train, test, y_train, y_test, StandardScaler(), 18, 
                max_features=0.3, min_samples_split=2, criterion = 'entropy')

train_poly, test_poly = create_poly(train, test, 2)

fit_predict(train_poly, test_poly, y_train, y_test, StandardScaler(), 18, 
                max_features=0.3, min_samples_split=2, criterion = 'entropy')


Additional feature engineering:
0.592857142857
0.617346938776


In [13]:
original_score = 0.514285714286
best_score = 0.625510204082
improvement = np.abs(np.round(100*(original_score - best_score)/original_score,2))
print('overall improvement is {} %'.format(improvement))

overall improvement is 21.63 %
